## Every Dream trainer

You will need your data prepared first before starting!  Don't waste rental fees if you're not ready to upload your files.  Your files should be captioned before you start with either the caption as the filename or in text files for each image alongside the image files.  See main README.md for more details. Tools are available to automatically caption your files.

[Instructions](https://github.com/victorchall/EveryDream-trainer/blob/main/README.md)

If you can sign up for Runpod here (shameless referral link): [Runpod](https://runpod.io?ref=oko38cd0)

If you are confused by the wall of text, join the discord here: [EveryDream Discord](https://discord.gg/uheqxU6sXN)

Make sure you have at least 40GB of Runpod **Volume** storage at a minimum so you don't waste training just 1 ckpt that is overtrained and have to start over.  Penny pinching on storage is ultimately a waste of your time and money!  This is setup to give you more than one ckpt so you don't overtrain.

### Starting model
Make sure you have your hugging face token ready to download the 1.5 mode. You can get one here: https://huggingface.co/settings/tokens
If you don't have a User Access Token, create one.  Or you can upload a starting checkpoint instead of using the HF download and skip that step, but you'll need to modify the starting model name when you start training (more info below).

In [1]:
# check that you have sufficient SWAP FILE space so saving CKPTs doesn't show "Killed"
!grep Swap /proc/meminfo
!swapon -s

SwapCached:           64 kB
SwapTotal:      18863096 kB
SwapFree:       18835756 kB
Filename				Type		Size	Used	Priority
/swap.img                              	file    	8388604	27340	-2
/dev/md2p1                             	partition	10474492	0	-3


In [2]:
!git clone https://github.com/socialhourmobile/EveryDream-trainer-sumtan.git
%cd EveryDream-trainer-sumtan

import codecs
finish_msg = codecs.encode('QBAR', 'rot_13')

print(finish_msg)

Cloning into 'everydream-trainer'...
remote: Enumerating objects: 1213, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 1213 (delta 69), reused 121 (delta 66), pack-reused 1085
Receiving objects: 100% (1213/1213), 50.30 MiB | 17.47 MiB/s, done.
Resolving deltas: 100% (710/710), done.
/workspace/everydream-trainer
DONE


## Install dependencies

**This will take a couple minutes.  Wait until it says "DONE" to move on.** 
You can ignore "warnings."

In [3]:
!pip install -q omegaconf
!pip install -q einops
!pip install -q pytorch-lightning==1.6.5
!pip install -q test-tube
!pip install -q transformers==4.19.2
!pip install -q kornia
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install -q setuptools==59.5.0
!pip install -q pillow==9.0.1
!pip install -q torchmetrics==0.6.0
!pip install -e .
!pip install huggingface_hub
!pip install ipywidgets==7.7.1
!pip install b2
!pip install requests
import ipywidgets as widgets
print(finish_msg)

Obtaining taming-transformers from git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
  Cloning https://github.com/CompVis/taming-transformers.git (to revision master) to ./src/taming-transformers
  Running command git clone -q https://github.com/CompVis/taming-transformers.git /workspace/everydream-trainer/src/taming-transformers
  Resolved https://github.com/CompVis/taming-transformers.git to commit 3ba01b241669f5ade541ce990f7650a3b8f65318
  Running setup.py develop for taming-transformers
Obtaining clip from git+https://github.com/openai/CLIP.git@main#egg=clip
  Cloning https://github.com/openai/CLIP.git (to revision main) to ./src/clip
  Running command git clone -q https://github.com/openai/CLIP.git /workspace/everydream-trainer/src/clip
  Resolved https://github.com/openai/CLIP.git to commit d50d76daa670286dd6cacf3bcd80b5e4823fc8e1
     |████████████████████████████████| 53 kB 2.2 MB/s eta 0:00:011
  Running setup.py develop for clip
Obtaining f

## Now that dependencies are installed, ready to move on!

## Log into huggingface
Run the cell below and paste your token into the prompt.  You can get your token from your [huggingface account page](https://huggingface.co/settings/tokens).

The token will not show on the screen, just press enter after you paste it.

Then run the following cell to download the base checkpoint (may take a minute).

In [4]:
from huggingface_hub import notebook_login

notebook_login()

b2_id = 004ba9b4c75d9210000000001
b2_key = K004x4Qv8pj9InJwj1VFqX5STJwubzE

!b2 authorize-account b2_id b2_key

In [ ]:
from huggingface_hub import hf_hub_download
downloaded_model_path = hf_hub_download(
 repo_id="runwayml/stable-diffusion-v1-5",
 filename="v1-5-pruned-emaonly.ckpt",
 use_auth_token=True
)
print(downloaded_model_path) # cache location
print(finish_msg)

# Upload training files

Ues the navigation on the left to open **"every-dream-trainer"** folder, then open **"input"** and upload your training files using the **up arrow button** above the file explorer.  

You can upload your images while you wait for the base checkpoint to download above, but do not continue until the checkpoint download is done.

If your captions are in .txt files, upload those in the same folder as the images. 

![upload](./demo/runpodupload.png)

You can check there are files in the folder by running the cell below (optional, just prints first 10).

In [ ]:
!ls -U input | head -10
# at least a few image filenames should show below, if not you uploaded to the wrong folder

## Tweak your YAML
You can adjust the YAML file to change the training parameters.  The v1-fine yamls are in the everydream-trainer/configs/stable-diffusion folder.  By default the "v1-finetune_runpod.yaml" is used here with some sane defaults for one subject with 20-30 images.

You can also upload your own in that folder and use it below.

Instructions are here: [EveryDream README](https://github.com/victorchall/EveryDream-trainer/blob/main/README.md) (hopefully you already read this?)

[Runpod YAML](configs/stable-diffusion/v1-finetune_runpod.yaml) is a good starting point for small datasets (30-50 images) and **is the default in the command below**. It will only keep 2 checkpoints.

If you are running on an A100 on Colab or otherwise, you can adjust the batch size up substantially.  Batch size 16 on A100 40GB as been tested as working.  In Colab, use the file navigation on the left to open the yaml.

If you need help with larger training, join the Discord for advice on making a custom yaml.

In [6]:
# change to use a custom yaml, but RUN THIS CELL NO MATTER WHAT to set your yaml path
my_yaml = "configs/stable-diffusion/v1-finetune_runpod.yaml"
# downloaded_model_path = "v1-5-pruned-emaonly.ckpt"  # this is the default, but you can change it by uncommenting if you uploaded a file into /everydream-trainer
print(downloaded_model_path) # reminder in case something went wrong with download
print(f"yaml set to: {my_yaml}")

/root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/33b6c600168fce4960d7cfb9c00a9cc895a21c55/sd-v1-5-inpainting.ckpt
yaml set to: configs/stable-diffusion/v1-finetune_everydream.yaml


## Run the trainer
Next cell runs training.  This will take a while depending on your number of images, repeats, and max_epochs.  

You can watch for test images in the logs folder.

In [ ]:
# run the trainer, wait until it finishes then SCROLL DOWN to the next cell
!python main.py --base {my_yaml} -t  --actual_resume {downloaded_model_path} -n test --data_root input
print(finish_msg)

Global seed set to 23
Running on GPUs 0,
Loading model from /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/33b6c600168fce4960d7cfb9c00a9cc895a21c55/sd-v1-5-inpainting.ckpt
ckpt: /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/33b6c600168fce4960d7cfb9c00a9cc895a21c55/sd-v1-5-inpainting.ckpt has 440000 steps
Instantiating config for: ldm.models.diffusion.ddpm.LatentDiffusion with config:
{'linear_start': 0.00085, 'linear_end': 0.012, 'num_timesteps_cond': 1, 'log_every_t': 200, 'timesteps': 1000, 'first_stage_key': 'image', 'cond_stage_key': 'caption', 'image_size': 64, 'channels': 4, 'cond_stage_trainable': True, 'conditioning_key': 'crossattn', 'monitor': 'val/loss_simple_ema', 'scale_factor': 0.18215, 'use_ema': False, 'unfreeze_model': True, 'model_lr': 1e-06, 'unet_config': {'target': 'ldm.modules.diffusionmodules.openaimodel.UNetModel', 'params': {'image_size': 32, 'in_channels': 4, 'out_channels': 4, 'model

## Prune your checkpoints
This will create 2GB pruned files for all your checkpoints and delete the 11GB files.

In [ ]:
# prune the ckpts
!python scripts/autoprune_all.py --delete

## Download your checkpoints

Use the cell below to generate links, right click and save as to download.  If you use Colab, you can skip this and use the Gdrive connect instead.

[EveryDream Discord](https://discord.gg/uheqxU6sXN)

In [ ]:
import glob
from IPython.display import FileLink


for f in glob.glob("*.ckpt"):
    !python scripts/upload_model.py --file_name {f}
    display(FileLink(f))
# right click save as to download the ckpt files

# Gdrive connect

For colab only, copies your ckpts to your gdrive.  If the EveryDreamCkpts folder already exists in your gdrive there will be an error, but it should still copy your files. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir /content/drive/MyDrive/EveryDreamCkpts
!cp *.ckpt /content/drive/MyDrive/EveryDreamCkpts